### Import Libraries

In [1]:
from Server_Model import *
from flask import Flask, json, request, render_template, session, redirect
from flask import send_file
from urllib.parse import urlparse
import requests
from random import randrange
from flask import session, url_for

### Flask APIs

In [2]:
def check_available_clients():
    
    try:
        with open("client.list", "r") as f:
            client_list = f.read()   
    except:
        client_list = ""
        
    client_list = client_list.split("\n")
    
    available_clients = []
    for c in client_list:
        r = requests.get(url = "http://"+c+'/active_test')
    
        #print(r, r.status_code, r.reason, r.text)
        if r.status_code == 200:
            #print("yeah")
            available_clients.append(c)
            
    return available_clients

In [ ]:
api = Flask(__name__)


@api.route('/', methods=['GET'])
def get_landing_page():
    
    try:
        with open("client.list", "r") as f:
            client_list = f.read()   
    except:
        client_list = ""
        
    client_list = client_list.split("\n")
    if len(client_list) > 0:
        return render_template('ServerLanding.html', list_clients=list(client_list))
    return "No clients"

@api.route('/register', methods=['GET'])
def register_client():
    
    port = request.args.get('port')
    url = request.args.get('url')
    
    try:
        with open("client.list", "r") as f:
            client_list = f.read()   
    except:
        client_list = ""
        
    client_list = client_list.split("\n")
    client_list.append(url + ":" + port)
    client_list = list(set(client_list))
    client_list = '\n'.join(str(x) for x in client_list)
    
    with open("client.list", "w") as f:
        f.write(client_list.strip())
    
    serverack = {'server_ack': '1'}
    return str(serverack)


@api.route('/send_weights_to_clients', methods=['GET'])
def send_weights_to_clients():
    
    model_aggregation()
    clients = check_available_clients()
        
    for c in clients:
        file = open("agg_model.npy", 'rb')
        data = {'fname':'agg_model.npy'}
        files = {
            'json': ('json_data', json.dumps(data), 'application/json'),
            'model': ('agg_model.npy', file, 'application/octet-stream')
        }

        #print(c+'/downloadmodel')
        req = requests.post(url="http://"+c+'/downloadmodel', files=files)
        #print(req.status_code)

    return "Aggregated model sent !"

@api.route('/client_model', methods=['POST'])
def getmodel():
    if request.method == 'POST':
        file = request.files['model'].read()
        fname = request.files['json'].read()

        fname = ast.literal_eval(fname.decode("utf-8"))
        cli = fname['url']+'_' + str(fname['port'])
        fname = cli +"_" + fname['fname']

        #print(fname)
        wfile = open("client_models_"+fname, 'wb+')
        wfile.write(file)
        wfile.close()
        
        return "Model received!"
    else:
        return "No file received!"

if __name__ == '__main__':
    api.secret_key = randrange(100000)
    api.config['SESSION_TYPE'] = 'filesystem'
    api.config.update(
        SESSION_COOKIE_NAME = 'session_server',
        SESSION_COOKIE_PATH = '/server/'
    )
    api.run(port=10011)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:10011/ (Press CTRL+C to quit)


10000 test samples
['client_models_127.0.0.1_5000_agg_model.npy']
client_models_127.0.0.1_5000_agg_model.npy


C:\Users\pk15425\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
127.0.0.1 - - [13/Jul/2022 15:36:12] "GET /send_weights_to_clients HTTP/1.1" 200 -


Test loss: 0.053886134177446365
Test accuracy: 0.9825000166893005
Model written to storage!


127.0.0.1 - - [13/Jul/2022 15:37:28] "POST /client_model HTTP/1.1" 200 -


10000 test samples
['client_models_127.0.0.1_5000_agg_model.npy', 'client_models_127.0.0.2_5000_agg_model - Copy.npy']
client_models_127.0.0.1_5000_agg_model.npy
client_models_127.0.0.2_5000_agg_model - Copy.npy


127.0.0.1 - - [13/Jul/2022 15:38:10] "GET /send_weights_to_clients HTTP/1.1" 200 -


Test loss: 0.04280697926878929
Test accuracy: 0.9865999817848206
Model written to storage!
